In [ ]:
import pandas as pd

def getStockDetails(name,isAnnual):
    sections = ['stock-financial-report','stock-financial-ratio']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    # headers = []
    quarters = []
    data = []
    toggled = False
    for section in sections:
            obj = {}
            keys = []
            # names = []
            headerElements = driver.find_element(By.CSS_SELECTOR,
        f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            # if len(headers) == 0:
            #     for div in BeautifulSoup(headerElements.get_attribute('innerHTML'), 'html.parser').find_all('div', {'class': 'topic'}):
            #         headers.append(div.get_text())
            if not toggled and isAnnual:
                toggled = True
                e = driver.find_element(By.CLASS_NAME,'toggle')
                webdriver.ActionChains(driver).move_to_element(e).click(e).perform()
            for topic in headerElements.find_elements(By.CLASS_NAME,'topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                keys.append(key)
                # name = BeautifulSoup(topic.get_attribute('innerHTML'), 'html.parser').get_text()
                # names.append(name)
                
            contentElements = driver.find_element(By.CSS_SELECTOR, 
                                f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]}"
                obj[key] = values
                data.append(obj)
    # return (headers,quarters,data)
    return (quarters,data)

def getFinancialDetails(symbol,isAnnual):
    # headers,quarterlyOrAnnual,responseData = getStockDetails(name=symbol,isAnnual=isAnnual)
    quarterlyOrAnnual,responseData = getStockDetails(name=symbol,isAnnual=isAnnual)
    data = {}
    for d in responseData:
        data = dict(list(data.items()) + list(d.items()))
    data['Period'] = quarterlyOrAnnual
    data['Symbol'] = symbol
    df = pd.DataFrame(data) 
    df = df.iloc[:-1]
    df = df.drop(df.columns[[5, 6, 8, 9, 10, 11, 15, 16, 17, 18, 19]], axis=1)
    return df

In [19]:
import requests
response = requests.get('https://www.finnomena.com/stock/AACI')

if response.status_code == 404:
    pass
else:
    print("Page exists")

In [23]:
import pandas as pd
import numpy as np

# create a sample dataframe with string representations of numbers and empty values
df = pd.DataFrame({'A': ['1.23', '2.34', '', '3.45', ''],
                   'B': ['4.56', '5.67', '6.78', '', ''],
                   'C': ['7.89', '8.90', '9.01', np.nan, '']})

# display the original dataframe
print("Original DataFrame:")
print(df)

# replace empty strings or NaN values with None
df = df.where(pd.notnull(df), None)

# convert columns 0 to 3 to float type and remove commas
df.iloc[:, :4] = df.iloc[:, :4].applymap(lambda x: str(x).replace(',', '')).astype(float)

# display the updated dataframe
print("\nDataFrame with converted float columns:")
print(df)


Original DataFrame:
      A     B     C
0  1.23  4.56  7.89
1  2.34  5.67  8.90
2        6.78  9.01
3  3.45         NaN
4                  


ValueError: could not convert string to float: ''

In [1]:
import pandas as pd

# create a list of 10 dataframes
df_list = [pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]}),
           pd.DataFrame({'A': [4, 5, 6], 'B': [7, 8, 9]}),
           pd.DataFrame({'A': [7, 8, 9], 'B': [10, 11, 12]}),
           pd.DataFrame({'A': [10, 11, 12], 'B': [13, 14, 15]}),
           pd.DataFrame({'A': [13, 14, 15], 'B': [16, 17, 18]}),
           pd.DataFrame({'A': [16, 17, 18], 'B': [19, 20, 21]}),
           pd.DataFrame({'A': [19, 20, 21], 'B': [22, 23, 24]}),
           pd.DataFrame({'A': [22, 23, 24], 'B': [25, 26, 27]}),
           pd.DataFrame({'A': [25, 26, 27], 'B': [28, 29, 30]}),
           pd.DataFrame({'A': [28, 29, 30], 'B': [31, 32, 33]})]

# concatenate the dataframes in the list
concatenated_df = pd.concat(df_list)

# print the concatenated dataframe
print(concatenated_df)

    A   B
0   1   4
1   2   5
2   3   6
0   4   7
1   5   8
2   6   9
0   7  10
1   8  11
2   9  12
0  10  13
1  11  14
2  12  15
0  13  16
1  14  17
2  15  18
0  16  19
1  17  20
2  18  21
0  19  22
1  20  23
2  21  24
0  22  25
1  23  26
2  24  27
0  25  28
1  26  29
2  27  30
0  28  31
1  29  32
2  30  33
